# Introduction
This notebook shows how to connect to Alpaca and receive live price data from it. We run a strategy using this live data feed for 10 minutes and then show some results.  

In [1]:
%use roboquant
Welcome()

roboquant (version 0.8-SNAPSHOT) 
 home: /home/peter/.roboquant 
 os: Linux 5.13.14-200.fc34.x86_64 
 jvm: OpenJDK 64-Bit Server VM 11.0.12 
 memory: 7946 MB 
 env: LAB

By default the Alpaca feed classes are not imported, so we need to do that first.

In [2]:
import org.roboquant.feeds.alpaca.*

# Setup
We now create, as usual, the strategy we want to test. There is nothing specific to Alapca that the strategy needs to be aware of. 
And we create an instance of Roboquant with the strategy we just defined and some metrics to later see how we have done. Again this is all independent of the data feed we are going to use.

In [3]:
val strategy = EMACrossover.shortTerm()
val roboquant =  Roboquant(strategy, AccountSummary(), ProgressMetric())

# Feed
Now we create an instance of the AlpacaFeed and subscribe to one or more assets. We need to provide the API Key and Secret to authenticate against the Alpaca API. We can do this by:

* Setting the environment or JVM variables APCA_API_KEY_ID and APCA_API_SECRET_KEY
* Or pass them as parameters to the *AlpacaFeed* constructor. If you take this approach, make sure you don't accidentally share your notebook with anyone.

Also if you are going to test this yourself, make sure you use KEY/SECRET for papertrading.


In [4]:
// val feed = AlpacaFeed("your api key", "you secret")

val feed = AlpacaFeed()

Alpaca offers the prices for many stocks. For this notebook we're going to limit that a bit, we just get all the stocks who's symboal name starts with "AA" and then subscribe to them. Alpaca has a generous API rate limit, so it is possible for example to subscribe to all S&P 500 assets.

In [5]:
val assets = feed.assets.filter { it.symbol.startsWith("AA") }
assets.summary()

Assets
├── STOCK AAA 
├── STOCK AAAU 
├── STOCK AAC.U 
├── STOCK AAL 
├── STOCK AAMC 
├── STOCK AAME 
├── STOCK AAN 
├── STOCK AAON 
├── STOCK AAP 
├── STOCK AAPL 
├── STOCK AAQC 
├── STOCK AAQC.U 
├── STOCK AAT 
├── STOCK AAU 
├── STOCK AAWW 
├── STOCK AAXJ 
├── STOCK AAC 
├── STOCK AACG 
├── STOCK AAIN 
├── STOCK AAOI 
├── STOCK AATC 
├── STOCK AACIU 
├── STOCK AADI 
├── STOCK AADR 
├── STOCK AAIC 
└── STOCK AA 


In [6]:
feed.subscribe(assets)

# Run
We have all the components we need to start the test. All that remains, is to start the run and evaluate the strategy against the feed. 

We'll run it for 10 minutes. If the time displayed looks wrong, don't forget that roboquant uses a timezone independent representation for all internal time processing (same as UTC).

In [7]:
val timeFrame = TimeFrame.nextMinutes(10)
roboquant.run(feed, timeFrame)
feed.disconnect()

2021-09-17 09:15:38 - 2021-09-17 09:25:38 | 100% |████████████████████████████████████████| Roboquant-0 | run=1 | phase=MAIN |


# Results
Lets see a few result

In [8]:
roboquant.broker.account.summary()

Account
├── last update: 2021-09-17T09:25:28.108537Z
├── base currency: USD
├── buying power: 1,000,000.00
├── total value: 1,000,000.00
├── Trades
│   ├── total: 0
│   ├── realized p&l: 0.00
│   ├── assets: 0
│   └── fee: 0.00
├── Orders
│   ├── total: 0
│   ├── open: 0
│   └── closed: 0
├── Portfolio
│   ├── open positions: 0
│   ├── long positions: 0
│   ├── short positions: 0
│   ├── estimated value: 0.00
│   └── unrealized p&l: 0.00
└── Cash
    └── USD: 1,000,000.00


In [12]:
val logger = roboquant.logger
logger.summary().print()

Metrics
└── 2021-09-17T09:25:28.108537Z
    ├── account.orders: 0
    ├── account.orders.open: 0
    ├── account.trades: 0
    ├── account.portfolio.assets: 0
    ├── account.cash.currencies: 1
    ├── account.cash.total: 1000000.000
    ├── account.buyingPower: 1000000.000
    ├── account.value: 1000000.000
    ├── account.change: .000
    ├── progress.actions: 0
    ├── progress.steps: 59
    ├── progress.trades: 0
    ├── progress.orders: 0
    └── progress.wallTime: 590043


In [13]:
logger.getMetricNames().summary()

Values
├── account.orders
├── account.orders.open
├── account.trades
├── account.portfolio.assets
├── account.cash.currencies
├── account.cash.total
├── account.buyingPower
├── account.value
├── account.change
├── progress.actions
├── progress.steps
├── progress.trades
├── progress.orders
└── progress.wallTime


In [10]:
val accountValue = logger.getMetric("account.value")
MetricChart(accountValue)